# Load CMIP6 Data with Intake ESM

[Intake ESM](https://intake-esm.readthedocs.io/en/latest/) is an experimental new package that aims to provide a higher-level interface to searching and loading Earth System Model data archives, such as CMIP6. The packages is under very active development, and features may be unstable. Please report any issues or suggestions [on github](https://github.com/NCAR/intake-esm/issues).

In [1]:
import xarray as xr
xr.set_options(display_style='html')
import intake
%matplotlib inline

Intake ESM works by parsing an [ESM Collection Spec](https://github.com/NCAR/esm-collection-spec/) and converting it to an [intake catalog](https://intake.readthedocs.io/en/latest). The collection spec is stored in a .json file. Here we open it using intake.

In [2]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

pangeo-cmip6-ESM Collection with 142406 entries:
	> 14 activity_id(s)

	> 24 institution_id(s)

	> 55 source_id(s)

	> 99 experiment_id(s)

	> 123 member_id(s)

	> 28 table_id(s)

	> 243 variable_id(s)

	> 8 grid_label(s)

	> 142406 zstore(s)

	> 60 dcpp_init_year(s)

We can now use intake methods to search the collection, and, if desired, export a pandas dataframe.

In [3]:
cat = col.search(experiment_id=['historical', 'ssp585'], table_id='Oyr', variable_id='o2',
                 grid_label='gn')
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
0,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
1,CMIP,CCCma,CanESM5,historical,r10i1p2f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r10i1...,NaN
2,CMIP,CCCma,CanESM5,historical,r11i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r11i1...,NaN
3,CMIP,CCCma,CanESM5,historical,r12i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r12i1...,NaN
4,CMIP,CCCma,CanESM5,historical,r13i1p1f1,Oyr,o2,gn,gs://cmip6/CMIP/CCCma/CanESM5/historical/r13i1...,NaN
...,...,...,...,...,...,...,...,...,...,...
88,ScenarioMIP,CCCma,CanESM5,ssp585,r9i1p1f1,Oyr,o2,gn,gs://cmip6/ScenarioMIP/CCCma/CanESM5/ssp585/r9...,NaN
89,ScenarioMIP,CCCma,CanESM5,ssp585,r9i1p2f1,Oyr,o2,gn,gs://cmip6/ScenarioMIP/CCCma/CanESM5/ssp585/r9...,NaN
90,ScenarioMIP,DKRZ,MPI-ESM1-2-HR,ssp585,r1i1p1f1,Oyr,o2,gn,gs://cmip6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp5...,NaN
91,ScenarioMIP,IPSL,IPSL-CM6A-LR,ssp585,r1i1p1f1,Oyr,o2,gn,gs://cmip6/ScenarioMIP/IPSL/IPSL-CM6A-LR/ssp58...,NaN


Intake knows how to automatically open the datasets using xarray. Furthermore, intake esm contains special logic to concatenate and merge the individual results of our query into larger, more high-level aggregated xarray datasets.

In [4]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True})
list(dset_dict.keys())

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 7 group(s)


['CMIP.CCCma.CanESM5.historical.Oyr.gn',
 'CMIP.IPSL.IPSL-CM6A-LR.historical.Oyr.gn',
 'CMIP.MIROC.MIROC-ES2L.historical.Oyr.gn',
 'ScenarioMIP.CCCma.CanESM5.ssp585.Oyr.gn',
 'ScenarioMIP.DKRZ.MPI-ESM1-2-HR.ssp585.Oyr.gn',
 'ScenarioMIP.IPSL.IPSL-CM6A-LR.ssp585.Oyr.gn',
 'ScenarioMIP.MIROC.MIROC-ES2L.ssp585.Oyr.gn']

In [5]:
ds = dset_dict['CMIP.CCCma.CanESM5.historical.Oyr.gn']
ds

<xarray.Dataset>
Dimensions:             (bnds: 2, i: 360, j: 291, lev: 45, member_id: 35, time: 165, vertices: 4)
Coordinates:
    latitude            (j, i) float64 dask.array<chunksize=(291, 360), meta=np.ndarray>
    lev_bnds            (lev, bnds) float64 dask.array<chunksize=(45, 2), meta=np.ndarray>
    time_bnds           (time, bnds) object dask.array<chunksize=(165, 2), meta=np.ndarray>
    longitude           (j, i) float64 dask.array<chunksize=(291, 360), meta=np.ndarray>
  * i                   (i) int32 0 1 2 3 4 5 6 ... 353 354 355 356 357 358 359
  * j                   (j) int32 0 1 2 3 4 5 6 ... 284 285 286 287 288 289 290
  * time                (time) object 1850-07-02 12:00:00 ... 2014-07-02 12:00:00
  * lev                 (lev) float64 3.047 9.454 16.36 ... 5.375e+03 5.625e+03
  * member_id           (member_id) <U9 'r10i1p1f1' 'r10i1p2f1' ... 'r9i1p2f1'
Dimensions without coordinates: bnds, vertices
Data variables:
    vertices_latitude   (j, i, vertices) float64 dask.array<chunksize=(291, 360, 4), meta=np.ndarray>
    vertices_longitude  (j, i, vertices) float64 dask.array<chunksize=(291, 360, 4), meta=np.ndarray>
    o2                  (member_id, time, lev, j, i) float32 dask.array<chunksize=(1, 12, 45, 291, 360), meta=np.ndarray>
Attributes:
    forcing_index:               1
    sub_experiment:              none
    institution:                 Canadian Centre for Climate Modelling and An...
    branch_method:               Spin-up documentation
    YMDH_branch_time_in_parent:  5950:01:01:00
    table_id:                    Oyr
    YMDH_branch_time_in_child:   1850:01:01:00
    initialization_index:        1
    license:                     CMIP6 model data produced by The Government ...
    grid:                        ORCA1 tripolar grid, 1 deg with refinement t...
    parent_experiment_id:        piControl
    table_info:                  Creation Date:(20 February 2019) MD5:374fbe5...
    branch_time_in_child:        0.0
    source_id:                   CanESM5
    version:                     v20190429
    grid_label:                  gn
    product:                     model-output
    experiment:                  all-forcing simulation of the recent past
    frequency:                   yr
    nominal_resolution:          100 km
    sub_experiment_id:           none
    further_info_url:            https://furtherinfo.es-doc.org/CMIP6.CCCma.C...
    history:                     2019-05-02T13:53:53Z ;rewrote data to be con...
    branch_time_in_parent:       1496500.0
    activity_id:                 CMIP
    CCCma_model_hash:            Unknown
    Conventions:                 CF-1.7 CMIP-6.2
    realm:                       ocnBgchem
    tracking_id:                 hdl:21.14100/41426118-701c-482b-ae16-82932e4...
    external_variables:          areacello volcello
    variant_label:               r9i1p2f1
    institution_id:              CCCma
    CCCma_parent_runid:          p2-pictrl
    parent_time_units:           days since 1850-01-01 0:0:0.0
    CCCma_runid:                 p2-his09
    cmor_version:                3.4.0
    variable_id:                 o2
    source_type:                 AOGCM
    creation_date:               2019-05-30T08:58:45Z
    experiment_id:               historical
    intake_esm_varname:          o2
    parent_activity_id:          CMIP
    source:                      CanESM5 (2019): \naerosol: interactive\natmo...
    mip_era:                     CMIP6
    data_specs_version:          01.00.29
    parent_source_id:            CanESM5
    status:                      2019-10-25;created;by nhn2@columbia.edu
    parent_mip_era:              CMIP6
    references:                  Geophysical Model Development Special issue ...
    realization_index:           9
    contact:                     ec.cccma.info-info.ccmac.ec@canada.ca
    title:                       CanESM5 output prepared for CMIP6